# Basic demonstration of Gemini SDK integration functionality with the Unity Catalog AI Toolkit SDK

To get started with this, you will need a Google API Key. 
For testing purposes, you can generate a new account, and use your evaluation test key (no credit card required!).

Once you have acquired your key, set it to the environment variable `GOOGLE_API_KEY` after storing it in the `Databricks Secrets` API (accessible via dbutils or the databricks sdk workspace client).

In [0]:
%pip install -Uqq unitycatalog-ai unitycatalog-gemini
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## Setting your API Key

Don't forget to remove the key after you're done running cell 4!

In [1]:
import base64
import os

from databricks.sdk import WorkspaceClient

workspace_client = WorkspaceClient(profile="optional to specify profile")

secret_scope = "puneet_jain"  # Change me!

# Run this if you don't have the API key set to your secrets scope yet

# if secret_scope not in [scope.name for scope in workspace_client.secrets.list_scopes()]:
#     workspace_client.secrets.create_scope(secret_scope)

# my_secret = "<your API key, temporarily>"

# workspace_client.secrets.put_secret(scope=secret_scope, key="gemini_api_key", string_value=my_secret)

## Fetch the key and set it to the environment variable key that the gemini SDK needs

In [2]:
os.environ["GOOGLE_API_KEY"] = base64.b64decode(
    workspace_client.secrets.get_secret(scope=secret_scope, key="gemini_api_key").value
).decode()

In [3]:
assert "GOOGLE_API_KEY" in os.environ, (
    "Please set the GOOGLE_API_KEY environment variable to your Gemini API key"
)

## Import the UC client for Databricks UC. 
This will allow for function creation through either the `create_function` API (requires the defined `sql_body` statement) or the `create_python_function` (requires a type-hint-applied and docstring commented python callable). 

In [ ]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from unitycatalog.ai.gemini.toolkit import UCFunctionToolkit

## Set the UC Catalog and Schema 

You must set both of these that you will be using to store and execute your function(s). If these do not exist, ensure that you create them first. 

In [5]:
CATALOG = "puneetjain_uc"  # Change me!
SCHEMA = "autogen_ucai"  # Change me if you want

In [ ]:
client = DatabricksFunctionClient(profile="optional to specify profile")

### Define a Callable
The requirements for the callable:

**typing**

Types **must** be supplied for both the arguments and the return type. Function signatures that do not have these defined will raise a `ValueError`.

The following types are not allowed:
`Union`
`Any`

Additional caveats:
Collections **must** supply typing of the interior components. For instance, ``typing.Dict`` is not allowed, but ``typing.Dict[str, str]`` will work correctly. 

**doc strings**

The doc string **must** be in the Google Docstring format.
Args and Returns comments are optional, but the function description **is required**. 

In [8]:
def fetch_weather(location: str) -> str:
    """
    Fetches the current weather in celsius for a given location.

    Args:
        location (str): The location to fetch the weather for.

    Returns:
        str: The current weather in celsius for the given location.
    """

    return "243.9 C"

In [9]:
client.create_python_function(func=fetch_weather, catalog=CATALOG, schema=SCHEMA, replace=True)

FunctionInfo(browse_only=None, catalog_name='puneetjain_uc', comment='Fetches the current weather in celsius for a given location.', created_at=1734574994989, created_by='puneet.jain@databricks.com', data_type=<ColumnTypeName.STRING: 'STRING'>, external_language='Python', external_name=None, full_data_type='STRING', full_name='puneetjain_uc.autogen_ucai.fetch_weather', function_id='1501f129-50de-400e-b07f-898fe1754bf5', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=0, comment='The location to fetch the weather for.', parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"location","type":"string","nullable":true,"metadata":{"comment":"The location to fetch the weather for."}}', type_precision=0, type_scale=0)]), is_deterministic=False, is_null_call=None, metastore_id='19a85dee-54bc-43

In [10]:
# Create a tool instance to use with Gemini

toolkit = UCFunctionToolkit(function_names=[f"{CATALOG}.{SCHEMA}.fetch_weather"], client=client)

tools = toolkit.generate_callable_tool_list()

## Submit the question

In the question request, submit the defined tools from the `UCFunctionToolkit` instance functions that have been fetched. 

In [11]:
# Interface with Gemini via their SDK
from google.generativeai import GenerativeModel

multi = "What's the weather in Nome, AK and in Death Valley, CA?"


model = GenerativeModel(model_name="gemini-2.0-flash-exp", tools=tools)

chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message("What's the weather in Nome, AK and in Death Valley, CA?")
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The weather in Nome, AK is 243.9 C, and the weather in Death Valley, CA is 243.9 C.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }
          ],
          "avg_lo

## Show Details of Tool Calls

In [12]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])  # noqa: T201
    print("-" * 80)  # noqa: T201

user -> [{'text': "What's the weather in Nome, AK and in Death Valley, CA?"}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'fetch_weather', 'args': {'location': 'Nome, AK'}}}, {'function_call': {'name': 'fetch_weather', 'args': {'location': 'Death Valley, CA'}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'fetch_weather', 'response': {'result': '{"format": "SCALAR", "value": "243.9 C"}'}}}, {'function_response': {'name': 'fetch_weather', 'response': {'result': '{"format": "SCALAR", "value": "243.9 C"}'}}}]
--------------------------------------------------------------------------------
model -> [{'text': 'The weather in Nome, AK is 243.9 C, and the weather in Death Valley, CA is 243.9 C.\n'}]
--------------------------------------------------------------------------------


## Manually Execute Function calls

In [16]:
from google.generativeai.types import content_types

from unitycatalog.ai.gemini.utils import generate_tool_call_messages, get_function_calls

history = []
question = "What's the weather in Nome, AK and in Death Valley, CA?"


content = content_types.to_content(question)
if not content.role:
    content.role = "user"

history.append(content)

response = model.generate_content(history)
while function_calls := get_function_calls(response):
    history, function_calls = generate_tool_call_messages(
        model=model, response=response, conversation_history=history
    )

    response = model.generate_content(history)

response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The weather in Nome, AK is 243.9 C and the weather in Death Valley, CA is 243.9 C.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }
          ],
          "avg_log